# Pryngles module: Science

In [6]:
from pryngles import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## External modules

In [7]:
import numpy as np
import math as mh
import spiceypy as spy
from scipy.integrate import quad
from scipy.spatial import ConvexHull

## The Science class

The Science class is a class with routines intended to perform a wide diversity of mathematical, physical and astronomical calculations.

In [8]:
class Science(PrynglesCommon):pass

### Template method

In [9]:
def template(foo=1):
    """
    Method

    Parameters:

        foo: type [units], default = 1:
            Description.

    Return:

        fee: type [units]:
            Desctiption.

    """
    return foo

Science.template=template

In [10]:
if IN_JUPYTER:
    def test_temp(self):
        
        #Test
        Science.template()
        """
        self.assertEqual(self.P.Nr,8,True)
        self.assertEqual(np.isclose([P.physics.wrot],
                                    [2*np.pi/PlanetDefaults.physics["prot"]],
                                    rtol=1e-7),
                         [True]*1)
        self.assertRaises(AssertionError,lambda:Observer(primary="Nada"))
        """
        
    class Test(unittest.TestCase):pass
    Test.test_temp=test_temp
    unittest.main(argv=['first-arg-is-ignored'],exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


### Cartesian to spherical

In [99]:
def spherical(xyz):
    """
    Transform cartesian coordinates into spherical coordinates

    Parameters:

        xyz: array (3):
            Cartesian coordinates

    Return:

        rqf: array (3):
            Spherical coordinates (r, theta, phi) where theta is azimutal angle and phi is 
            elevation (complement of polar angle).                

            Notice that this convention is different than that of regular vectorial calculus
            where spherical coordinates are (r,theta,phi), but theta is the polar angle and phi 
            the ezimutal one.

    """
    r,theta,phi=spy.reclat(np.array(xyz))
    theta=2*mh.pi+theta if theta<0 else theta

    return np.array([r,theta,phi])

def cospherical(xyz):
    """Transform cartesian coordinates into cosine/sine of spherical angles
    
    Parameters:

        xyz: array (3):
            Cartesian coordinates
            
    Return:

        cqsqcf: array (3):
            Cosine/sine of spherical angles (cos theta, sin theta, cos phi) where theta is 
            azimutal angle and phi is elevation (complement of polar angle).         
    """
    rho=(xyz[0]**2+xyz[1]**2)**0.5
    sf=xyz[2]/(rho**2+xyz[2]**2)**0.5
    cq=xyz[0]/rho if not mh.isclose(rho,0) else 1
    sq=xyz[1]/rho if not mh.isclose(rho,0) else 0
    return np.array([cq,sq,sf])

def pcylindrical(xyz):
    """Transform cartesian coordinates into pseudo cylindrical coordinates
    
    Parameters:

        xyz: array (3):
            Cartesian coordinates
            
    Return:

        rhoazcf: array (3):
            Cylindrical coordinates expresed as rho, phi (azimutal angle) and cos(theta) (cosine
            of polar angle).    
    """
    rho=(xyz[0]**2+xyz[1]**2)**0.5
    r=(xyz[2]**2+rho**2)**0.5
    phi=mh.atan2(xyz[1],xyz[0])
    phi=phi if phi>0 else 2*np.pi+phi
    cost=xyz[2]/r if not mh.isclose(r,0) else mh.copysign(1,xyz[2])
    return np.array([rho,phi,cost])

def cartesian(rqf):
    """
    Transform cartesian coordinates into spherical coordinates

    Parameters:

        xyz: array (3):
            Cartesian coordinates

    Return:

        rqf: array (3):
            Spherical coordinates (r, theta, phi) where theta is azimutal angle and phi is 
            elevation (complement of polar angle).                

            Notice that this convention is different than that of regular vectorial calculus
            where spherical coordinates are (r,theta,phi), but theta is the polar angle and phi 
            the ezimutal one.

    """
    return spy.latrec(rqf[0],rqf[1],rqf[2])

Science.spherical=spherical
Science.cartesian=cartesian
Science.cospherical=cospherical
Science.pcylindrical=pcylindrical

In [100]:
if IN_JUPYTER:
    def test_coords(self):
        
        #Test axis
        axis=[
            [+1,+0,+0],[-1,+0,+0],
            [+0,+1,+0],[+0,-1,+0],
            [+0,+0,+1],[+0,+0,-1],
        ]
        for i,xyz in enumerate(axis):
            rqf=Science.spherical(xyz)
            cqsqsf=Science.cospherical(xyz)
            rhofct=Science.pcylindrical(xyz)
            print(f"Axis {i+1}:")
            print("\tSpherical:",rqf[0],rqf[1]*Consts.rad,rqf[2]*Consts.rad)
            print("\tCospherical:",cqsqsf)
            print("\tVerify:",mh.cos(rqf[1]),mh.sin(rqf[1]),mh.sin(rqf[2]))
            print("\tPseudo cilyndrical:",rhofct)
            print("\tVerify:",rqf[0]*np.cos(rqf[2]),rqf[1],mh.sin(rqf[2]))

        #Test spherical
        octants=[
            [+1,+1,+1],[-1,+1,+1],[-1,-1,+1],[+1,-1,+1],
            [+1,+1,-1],[-1,+1,-1],[-1,-1,-1],[+1,-1,-1]
        ]
        for i,xyz in enumerate(octants):
            rqf=Science.spherical(xyz)
            cqsqsf=Science.cospherical(xyz)
            rhofct=Science.pcylindrical(xyz)
            print(f"Octant {i+1}:")
            print("\tSpherical:",rqf[0],rqf[1]*Consts.rad,rqf[2]*Consts.rad)
            print("\tCospherical:",cqsqsf)
            print("\tVerify:",mh.cos(rqf[1]),mh.sin(rqf[1]),mh.sin(rqf[2]))
            print("\tPseudo cilyndrical:",rhofct)
            print("\tVerify:",rqf[0]*np.cos(rqf[2]),rqf[1],mh.sin(rqf[2]))
            
        #Test cartesian
        octants=[
            [1,45*Consts.deg,45*Consts.deg],[1,135*Consts.deg,45*Consts.deg],
            [1,225*Consts.deg,45*Consts.deg],[1,315*Consts.deg,45*Consts.deg],
            [1,45*Consts.deg,-45*Consts.deg],[1,135*Consts.deg,-45*Consts.deg],
            [1,225*Consts.deg,-45*Consts.deg],[1,315*Consts.deg,-45*Consts.deg]
        ]
        for i,rqf in enumerate(octants):
            xyz=Science.cartesian(rqf)
            print(f"Octant {i+1}:",xyz) 

    class Test(unittest.TestCase):pass
    Test.test_coords=test_coords
    unittest.main(argv=['first-arg-is-ignored'],exit=False)

.

Axis 1:
	Spherical: 1.0 0.0 0.0
	Cospherical: [1. 0. 0.]
	Verify: 1.0 0.0 0.0
	Pseudo cilyndrical: [1.         6.28318531 0.        ]
	Verify: 1.0 0.0 0.0
Axis 2:
	Spherical: 1.0 180.0 0.0
	Cospherical: [-1.  0.  0.]
	Verify: -1.0 1.2246467991473532e-16 0.0
	Pseudo cilyndrical: [1.         3.14159265 0.        ]
	Verify: 1.0 3.141592653589793 0.0
Axis 3:
	Spherical: 1.0 90.0 0.0
	Cospherical: [0. 1. 0.]
	Verify: 6.123233995736766e-17 1.0 0.0
	Pseudo cilyndrical: [1.         1.57079633 0.        ]
	Verify: 1.0 1.5707963267948966 0.0
Axis 4:
	Spherical: 1.0 270.0 0.0
	Cospherical: [ 0. -1.  0.]
	Verify: -1.8369701987210297e-16 -1.0 0.0
	Pseudo cilyndrical: [1.         4.71238898 0.        ]
	Verify: 1.0 4.71238898038469 0.0
Axis 5:
	Spherical: 1.0 0.0 90.0
	Cospherical: [1. 0. 1.]
	Verify: 1.0 0.0 1.0
	Pseudo cilyndrical: [0.         6.28318531 1.        ]
	Verify: 6.123233995736766e-17 0.0 1.0
Axis 6:
	Spherical: 1.0 0.0 -90.0
	Cospherical: [ 1.  0. -1.]
	Verify: 1.0 0.0 -1.0
	Pseudo ci


----------------------------------------------------------------------
Ran 1 test in 0.017s

OK


In [85]:
def rotation_matrix(ez,alpha):
    """
    Set a rotation matrix from the direction of the ez vector and a rotation angle alpha
    
    Parameter:
        ez: array (3)
            vector in the direction of the z-axis. 
            
        alpha: float (3) [rad]
            Rotation angle of the x-axis around z-axis (clockwise)
            
    Return:
        Msys2uni: float (3x3)
            Rotation matrix from the system defined by ez and the universal system.
            
        Muni2sys: float (3x3)
            Rotation matrix from the universal system to the system defined by ez
    """
    ez,one=spy.unorm(ez)
    ex=spy.ucrss([0,0,1],ez) #Spice is 5 faster for vcrss
    if spy.vnorm(ex)==0:
        ex=np.array([1,0,0]) if np.sum(ez)>0 else np.array([-1,0,0])
    ey=spy.ucrss(ez,ex)
    Msys2uni=np.array(list(np.vstack((ex,ey,ez)).transpose())).reshape((3,3))
    Muni2sys=spy.invert(Msys2uni)
    verbose(VERB_VERIFY,"Rotation axis:",ex,ey,ez)
    return Msys2uni,Muni2sys

Science.rotation_matrix=rotation_matrix

In [9]:
if IN_JUPYTER:
    def test_rot(self):
        
        Verbose.VERBOSITY=VERB_ALL
        
        #Test rotation
        Msys2uni,Muni2sys=Science.rotation_matrix([0,0,1],0)
        print(Msys2uni)

        Msys2uni,Muni2sys=Science.rotation_matrix([0,0,-1],0)
        print(Msys2uni)

        Verbose.VERBOSITY=VERB_NONE

    class Test(unittest.TestCase):pass
    Test.test_rot=test_rot
    unittest.main(argv=['first-arg-is-ignored'],exit=False)

.

      VERB3::rotation_matrix:: Rotation axis: [1 0 0] [0. 1. 0.] [0. 0. 1.]
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
      VERB3::rotation_matrix:: Rotation axis: [-1  0  0] [0. 1. 0.] [ 0.  0. -1.]
[[-1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0. -1.]]



----------------------------------------------------------------------
Ran 1 test in 0.119s

OK


In [10]:
LIMB_NORMALIZATIONS=dict()

In [11]:
def limb_darkening(rho,cs=[0.6562],N=None):
    """
    Parameters:
        rho: float:
            Distance to center of the star in units of stellar radius.
            
        cs: list, default = [0.6562]:
            List of limb darkening coefficients.
            
        N: float, default = 1:
            Normalization constant.
            
    Return:
        I: float:
            Normalized intensity of the star at rho.
    
    Notes: 
        Models in: https://pages.jh.edu/~dsing3/David_Sing/Limb_Darkening.html
        Coefficients available at: https://pages.jh.edu/~dsing3/LDfiles/LDCs.CoRot.Table1.txt

    Test code:
    
        fig=plt.figure()
        ax=fig.gca()
        rhos=np.linspace(0,1,100)
        Rs=1
        coefs=[0.6550]
        N=Util.limbDarkeningNormalization(coefs)
        ax.plot(rhos,Util.limbDarkening(rhos,Rs,coefs,N))
        coefs=[0.6022,0.0654]
        N=Util.limbDarkeningNormalization(coefs)
        ax.plot(rhos,Util.limbDarkening(rhos,Rs,coefs,N))
        coefs=[0.9724,-0.4962,0.2029]
        N=Util.limbDarkeningNormalization(coefs)
        ax.plot(rhos,Util.limbDarkening(rhos,Rs,coefs,N))        
    """
    mu=(1-rho**2)**0.5
    order=len(cs)
    
    #Calculate normalization constant
    if N is None:
        chash=hash(tuple(cs))
        if chash in LIMB_NORMALIZATIONS:
            N=LIMB_NORMALIZATIONS[chash]
        else:
            integrand=lambda rho:Science.limb_darkening(rho,cs,N=1)*2*np.pi*rho
            N=quad(integrand,0.0,1.0,epsrel=1e-5)[0]
            verbose(VERB_VERIFY,f"Normalization of limb darkening function for cs = {cs}, N = {N}")
            LIMB_NORMALIZATIONS[chash]=N
            
    if order==0:
        I=np.ones_like(rho)
    elif order==1:
        I=1-cs[0]*(1-mu)
    elif order==2:
        I=1-cs[0]*(1-mu)-cs[1]*(1-mu)**2
    elif order==3:
        I=1-cs[0]*(1-mu)-cs[1]*(1-mu**1.5)-cs[2]*(1-mu**2)
    elif order==4:
        I=1-cs[0]*(1-mu**0.5)-cs[1]*(1-mu)-cs[2]*(1-mu**1.5)-cs[3]*(1-mu**2)
    else:
        raise ValueError(f"Limb darkening not implemented for order {order}")
    return I/N

Science.limb_darkening=limb_darkening

In [12]:
if IN_JUPYTER:
    def test_limb(self):
        
        Verbose.VERBOSITY=VERB_ALL

        cs=[np.random.rand()]
        I=Science.limb_darkening(0.8,cs)
        print(I)
        
        fig=plt.figure()
        ax=fig.gca()

        rhos=np.linspace(0,1,100)
        coefs=[0.6550]
        ax.plot(rhos,Science.limb_darkening(rhos,coefs))
        coefs=[0.6022,0.0654]
        ax.plot(rhos,Science.limb_darkening(rhos,coefs))
        coefs=[0.9724,-0.4962,0.2029]
        ax.plot(rhos,Science.limb_darkening(rhos,coefs))    
        coefs=[-0.2018,2.1000,-2.0247,0.7567]
        ax.plot(rhos,Science.limb_darkening(rhos,coefs))
        Plot.pryngles_mark(ax)
        
        Verbose.VERBOSITY=VERB_NONE

    class Test(unittest.TestCase):pass
    Test.test_limb=test_limb
    unittest.main(argv=['first-arg-is-ignored'],exit=False)

      VERB3::limb_darkening:: Normalization of limb darkening function for cs = [0.3359221208038803], N = 2.7898158312912007
0.31028254337413047


<IPython.core.display.Javascript object>

.

      VERB3::limb_darkening:: Normalization of limb darkening function for cs = [0.655], N = 2.45567825755602
      VERB3::limb_darkening:: Normalization of limb darkening function for cs = [0.6022, 0.0654], N = 2.4767269283350704
      VERB3::limb_darkening:: Normalization of limb darkening function for cs = [0.9724, -0.4962, 0.2029], N = 2.472665297844586
      VERB3::limb_darkening:: Normalization of limb darkening function for cs = [-0.2018, 2.1, -2.0247, 0.7567], N = 2.6067005991229735



----------------------------------------------------------------------
Ran 1 test in 0.080s

OK


In [15]:
def points_in_hull(p, hull, tol=1e-12):
    """Determine if a set of points are inside a convex hull.
    
    Parameters:
        
        p: numpy array (Nx2):
            Set of coordinates for points to evaluate.
    
        hull: ConvexHull:
            Convex hull to evaluate.
            
    Return:
    
        inside: boolean array (N):
            Boolean array telling if points are inside the convex hull.
            
    Examples:
        
        import numpy as np
        
        rng = np.random.default_rng()
        points = rng.random((30, 2))
        hull = ConvexHull(points)
        
        ps = rng.random((30, 2))-0.5
        cond=points_in_hull(ps,hull)

        import matplotlib.pyplot as plt
        
        for simplex in hull.simplices:
            plt.plot(points[simplex, 0], points[simplex, 1], 'k-')

        for p in ps[cond]:
            plt.plot(p[0],p[1],'r*')

        for p in ps[~cond]:
            plt.plot(p[0],p[1],'co')
            
            
    Notes:
        Taken from https://stackoverflow.com/a/72483841
    """
    return np.all(hull.equations[:,:-1] @ p.T + np.repeat(hull.equations[:,-1][None,:], len(p), axis=0).T <= tol, 0)

Science.points_in_hull=points_in_hull

In [23]:
if IN_JUPYTER:
    def test_hull(self):
        
        Verbose.VERBOSITY=VERB_ALL

        rng = np.random.default_rng()
        points = rng.random((30, 2))
        hull = ConvexHull(points)

        ps = rng.random((30, 2))-0.5
        cond=Science.points_in_hull(ps,hull)
        print(len(cond),len(ps))

        import matplotlib.pyplot as plt
        plt.figure()
        for simplex in hull.simplices:
            plt.plot(points[simplex, 0], points[simplex, 1], 'k-')

        for p in ps[cond]:
            plt.plot(p[0],p[1],'r*')

        for p in ps[~cond]:
            plt.plot(p[0],p[1],'co')
        
        Verbose.VERBOSITY=VERB_NONE

    class Test(unittest.TestCase):pass
    Test.test_hull=test_hull
    unittest.main(argv=['first-arg-is-ignored'],exit=False)

30 30


<IPython.core.display.Javascript object>

.
----------------------------------------------------------------------
Ran 1 test in 0.037s

OK


--End--